In [1]:
import numpy as np
from PIL import Image
import math

In [2]:
canvas_radius = 4
canvas_size = 2 * canvas_radius + 1
canvas_dimension = (canvas_size, canvas_size)

In [3]:
multiplier = 2
lightmap_fn = lambda sqrd_dist: max(0, min(-(multiplier*(sqrd_dist))+18,14))

In [4]:
def reduce(arr, fn, start_val = None):
    if start_val is None:
        if len(arr) == 0:
            accum = start_val
        else:
            accum = arr[0]
            arr = arr[1:]
    else:
        accum = start_val
    for i in arr:
        accum = fn(accum, i)
    return accum

In [5]:
def dist(start, end):
    if len(start) > len(end):
        end += [0] * (len(start) - len(end))
    else:
        start += [0] * (len(end) - len(start))
    assert len(end) == len(start)
    distances = [start[i] - end[i] for i in range(len(start))]
    distances = [i**2 for i in distances]
    return math.sqrt(reduce(distances, lambda a,b:a+b))

def sqrd_dist(start, end):
    if len(start) > len(end):
        end += [0] * (len(start) - len(end))
    else:
        start += [0] * (len(end) - len(start))
    assert len(end) == len(start)
    distances = [start[i] - end[i] for i in range(len(start))]
    distances = [i**2 for i in distances]
    return reduce(distances, lambda a,b:a+b)

In [6]:
delta_x = 0.2
delta_y = 0
delta_fn = lambda x,y: (x + delta_x, y + delta_y)
x, y = 0, canvas_radius

def generate_image(start_x, start_y, canvas_dimension, delta_fn, lightmap_fn, dist_fn):
    while(start_x < canvas_dimension[0] and start_y < canvas_dimension[1]):
        m = np.zeros(canvas_dimension, np.uint8)
        for y in range(canvas_dimension[0]):
            for x in range(canvas_dimension[1]):
                distance = dist_fn([start_x, start_y], [x, y])
                light_level = lightmap_fn(distance)
                m[y][x] = math.floor(light_level / 16 * 255)
        yield m
        start_x, start_y = delta_fn(start_x, start_y)

In [7]:
imgs = map(Image.fromarray, generate_image(x, y, canvas_dimension, delta_fn, lightmap_fn, sqrd_dist))
imgs = map(lambda im: im.resize((canvas_size * 16, canvas_size * 16), Image.NEAREST), imgs)
im = next(imgs)
im.save('out.gif', save_all=True, append_images=imgs, loop = 0)

In [ ]:
# Best for radius 7
factor_fn = lambda x: max(0, min(1, x * -0.03 + 1.08))

# Best for radius 3
factor_fn = lambda x: max(0, min(.75, x * -0.27 + 1.1))


In [103]:
radius = 7
padding = 2
img_size = (radius + padding, radius + padding)
center = (radius + padding) // 2

sqrd_dist = lambda x, y: (center - x)**2 + (center - y)**2
factor_fn = lambda x: max(0, min(.75, x * -0.27 + 1.1))
# factor_fn = lambda x: max(0, min(1, 1.5 - x**(1/2) * .25))


max_light_level = 15.0

matrices = []
reverse = False

for dz in np.arange(-.5, .6, .1):
    for dx in np.arange(-.5, .6, .1) if reverse else np.arange(.5, -.6, -.1):
        ix = padding
        iy = padding
        m = np.zeros(img_size, dtype=float)
        while(ix < img_size[0] - padding):
            while(iy < img_size[0] - padding):
                dist = sqrd_dist(ix + dx, iy + dz)
                factor = factor_fn(dist)
                light_level = factor * max_light_level
#                 print(f"ix {ix}, iy {iy}, light {light_level}")
                m[iy][ix] = light_level / 15 * 255
                iy += 1
            ix += 1
            iy = padding
        matrices.append(m)
        reverse = not reverse
        
imgs = map(Image.fromarray, matrices)
imgs = map(lambda im: im.resize((canvas_size * 16, canvas_size * 16), Image.NEAREST), imgs)
im = next(imgs)
im.save('out.gif', save_all=True, append_images=imgs, loop = 0, duration=6)

In [91]:
factor_fn(15 * 15)

0

In [72]:
1.5-25**(1/2) * .12

0.9

In [97]:
len(matrices) / 20

6.05